In [1]:
import torch
import torch.nn.functional as F

***相同点***

* nn.Xxx和nn.functional.xxx的实际功能是相同  
nn.Conv2d和nn.functional.conv2d 都是进行卷积，      
nn.Dropout 和nn.functional.dropout都是进行dropout 
* 运行效率也是近乎相同。  


***不同点***

* nn.functional.xxx是函数接口，nn.Xxx是nn.functional.xxx的类封装

从代码分析 nn.Xxx 的forward 调用nn.functional.xxx 函数

    class Conv1d(_ConvNd):
        def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                     padding=0, dilation=1, groups=1, bias=True):
            kernel_size = _single(kernel_size)
            stride = _single(stride)
            padding = _single(padding)
            dilation = _single(dilation)
            super(Conv1d, self).__init__(
                in_channels, out_channels, kernel_size, stride, padding, dilation,
                False, _single(0), groups, bias)

        def forward(self, input):
            return F.conv1d(input, self.weight, self.bias, self.stride,
                            self.padding, self.dilation, self.groups)

    def conv1d(input, weight, bias=None, stride=1, padding=0, dilation=1,
               groups=1):
        if input is not None and input.dim() != 3:
            raise ValueError("Expected 3D tensor as input, got {}D tensor instead.".format(input.dim()))

        f = ConvNd(_single(stride), _single(padding), _single(dilation), False,
                   _single(0), groups, torch.backends.cudnn.benchmark,
                   torch.backends.cudnn.deterministic, torch.backends.cudnn.enabled)
        return f(input, weight, bias)


* 两者的调用方式不同  
     nn.Xxx 需要先实例化   
     nn.functional.xxx同时传入输入数据和weight, bias等其他参数 。

* nn.Xxx继承于nn.Module， 能够很好的与nn.Sequential结合使用， 而nn.functional.xxx无法与nn.Sequential结合使用。

* nn.Xxx不需要你自己定义和管理weight；而nn.functional.xxx需要你自己定义weight，每次调用的时候都需要手动传入weight, 不利于代码复用。

*使用建议 ：  

**能使用nn.Xxx情况下尽量使用，不行再换nn.functional.xxx**
* conv2d, linear, batch_norm 使用nn.Xxx
* maxpool, loss func, activation func 使用nn.functional.xxx

**dropout，推荐使用nn.Xxx** 